## Problem Set 2: Merging and regular expressions

**Total points (without extra credit)**: 30 

**Background on the policy context**: here, we're going to use two datasets to practice reshaping, merging, and regular expression patterns. Both datasets relate to the broader issue of which employers might be violating the rights of temporary guestworkers granted visas under the H-2A program. Here are some articles about potential exploitation of guestworkers by firms and inequality caused by minimal oversight:

- News media coverage of labor abuses of temporary guestworkers: https://www.buzzfeednews.com/article/kenbensinger/the-pushovers 
- GAO report on labor abuses of temporary guestworkers: https://www.gao.gov/products/gao-15-154

The following datasets are located in `pset2_inputdata` (need to unzip): 

- `jobs_clean`: a dataset of guestworker jobs posted by many employers, some of whom have been debarred (banned) from the program for labor abuses; others not debarred
- `debar`: a dataset of employers who committed violations of labor regulations meant to protect temporary guestworkers 


You can view a codebook here: https://docs.google.com/spreadsheets/d/1rF9GJEC8pPKxipD0TsoG9DVdqz3EJ-b-BHEtyioAX7I/edit?usp=sharing


In [107]:
## helpful packages
import pandas as pd
import numpy as np
import random
import re
import os
from plotnine import *
import plotnine

## repeated printouts
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"


# 1. Reshaping data (13 points total)

Load the following dataset stored in `pset2_inputdata`: `debar.csv`

This represents employers temporarily banned from hiring workers (debar.csv); call this `debar`


View the head()


In [3]:
debar = pd.read_csv("debar.csv")

debar.head()

,Name,"City, State",Violation,Duration,Start date,End date
0,J&J Harvesting,"Leads, ND",Failure to respond to audit (partial response),2 years,1/19/2014,1/18/2016
1,"Stahlman Apiaries, Inc","Selby, SD",Failure to respond to audit (partial response),1 year,2/19/2015,2/14/2016
2,Trust Nursery,"Pulaski, NY",Failure to respond to audit (partial response),1 year,3/21/2014,3/20/2015
3,Anton Fertilizer Inc.,"Dighton, KS",Failure to respond to audit (no response),2 years,3/30/2014,3/29/2016
4,"Great Plains Fluid Service, Inc.","Greensburg, KS",Failure to respond to audit (no response),2 years,3/30/2014,3/29/2016


## 1.1 (1 point)

Print the number of rows in `debar` versus the number of unique employer names (`Name`). Is there one row per employer or multiple rows for some employers?

In [4]:
# Number of rows in the dataset
num_rows = len(debar)

# Number of unique employer names
num_unique_employers = debar["Name"].nunique()

print(num_rows)
print(num_unique_employers)

114
98


## 1.2 Investigating duplicated rows (2 points)

A. Create a new column in `debar`--`is_repeated`-- that tells us whether an employer (`Name`) is repeated > 1 times

*Hint*: there are multiple ways to solve this but some possibilities to get the list of names that are repeated are:
- Using value_counts() on the `Name` variable and extracting the index from that value counts 
- Using groupby to count the rows attached to one name

B. Print the rows where `is_repeated == True` and interpret

C. Subset to the rows where `is_repeated == True` and save that data as `mult_debar`. Print the head() and shape

In [22]:
## Part A

# List of number of times an employer name is repeated
num_employers = debar["Name"].value_counts()

# List of employer names that are repeated
repeated_employee_list = num_employers[num_employers > 1].index

# Create the "is_repeated" column 
debar["is_repeated"] = debar["Name"].isin(repeated_employee_list)


## Part B

# print rows where "is_repeated"==True
debar[debar["is_repeated"] == True]

### Interpret!!!


# Correct typos in City, State
debar["City, State"] = debar["City, State"].str.replace("Utah", "UT") \
    .str.replace("Georgia", "GA") \
    .str.replace("Texas", "TX") \
    .str.replace("Altheimer, AK", "Altheimer, AR") \
    .str.replace("Brownsville, TX", "Brownfield, TX")

## Part C

# Subset to rows where "is_repeated" == True
mult_debar=debar[debar["is_repeated"] == True]
mult_debar.shape



,Name,"City, State",Violation,Duration,Start date,End date,is_repeated
6,Annabella Land & Cattle,"Annabella, UT",Non Payment,1 year,5/9/2014,5/9/2015,True
7,Autumn Hill Orchard,"Groton, MA",Failure to respond to audit (no response),2 years,7/6/2014,7/5/2016,True
8,"Caddo Creek Ranch, dba Paradise Ranch","Caddo, TX",Failure to respond to audit (partial response),2 years,7/20/2014,7/19/2016,True
11,Loewen Harvesting LLC,"Brownfield, TX",Failure to respond to audit (partial response),1 year,8/20/2014,8/19/2015,True
12,Rollo Farm Labor Contractor,"Miami, FL",Failure to respond to audit (no response),2 years,8/23/2014,8/22/2016,True
14,Sharon Mathis,"Tifton, GA",Failure to respond to audit (no response),2 years,11/16/2014,11/15/2016,True
15,SRT Farms,"Morton, TX",Failure to respond to audit (no response),2 years,11/16/2014,11/15/2016,True
16,Mark Duncan,"Roosevelt, UT",Failure to respond to audit (no response),2 years,11/16/2014,11/15/2016,True
17,"Maple Ridge Custom Services, LLC","Altheimer, AR",Failure to respond to audit (partial response),2 years,11/16/2014,11/15/2016,True
18,F&W Farms,"Ingalls, KS",Failure to respond to audit (partial response),2 years,12/10/2014,12/9/2016,True


(32, 7)

## 1.3 Reshape mult_debar to wide to begin filtering out duplicates (4 points)

You want to separate out two cases:

- Cases where the repeat rows for one employer are due to duplicated data 
- Cases where the repeat rows for one employer represent repeated violations for different issues

There are various ways to check duplicates in this data (eg converting `Violation` to lowercase; replacing spelled-out states with two-dig state codes)

We're going to use the simple rule of:

- A row is a duplicate if, within an employer (defined by Name + City, State), the Start date for each row's violation is the same 

To begin to check this, reshape `mult_debar` to a wide dataframe (`mult_debar_wide`) with the following columns, treating the `Name` and `City, State` as the index for the pivot:

- Name
- City, State
- start_date_viol1
- start_date_viol2

Print the head and shape

In [19]:
# Add a new column - violnum to differentiate between the two rows for each repeated name
mult_debar["violnum"] = mult_debar.groupby(["Name"]).cumcount() + 1

# Pivot the df
mult_debar_wide = mult_debar.pivot(index=["Name", "City, State"], columns="violnum", values="Start date")

# Rename the columns
mult_debar_wide = mult_debar_wide.rename(columns={1: "start_date_viol1", 2: "start_date_viol2"})

# Reset the index
mult_debar_wide.reset_index(inplace=True)

# Print the head and shape
mult_debar_wide.head()
mult_debar_wide.shape





/var/folders/0f/r6sb0yjx70l_25cgtvyjjqgr0000gn/T/ipykernel_69596/3327245051.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  mult_debar["violnum"] = mult_debar.groupby(["Name"]).cumcount() + 1


violnum,Name,"City, State",start_date_viol1,start_date_viol2
0,Annabella Land & Cattle,"Annabella, UT",5/9/2014,5/9/2014
1,Autumn Hill Orchard,"Groton, MA",7/6/2014,7/6/2014
2,"Caddo Creek Ranch, dba Paradise Ranch","Caddo, TX",7/20/2014,7/20/2014
3,Cisco Produce Inc.,"Cairo, GA",12/10/2014,12/10/2015
4,Dove Creek Farms,"Mount Vernon, TX",2/9/2018,2/9/2018


(16, 4)

## 1.4 Filter out duplicates from original debar data (6 points)

A. Using `mult_debar_wide`, add a column `is_dup` that takes value of True for cases where start_date_viol1 == start_date_viol2 marking the row as a duplicate

B. Going back to the original long-format data you loaded at the beginning- `debar`
    - For employers where `is_dup == True` as indicated by your wide-format dataframe, only keep `violnum == viol1`
    - For all other employers (so is_dup == False and ones we didnt need to check duplicates for), keep all violnum
    - Remove the `is_repeated` column from the `debar` data

**Hint**: you can complete part B without a for loop; `pd.concat` with axis = 0 (row binding) is one way

Call the resulting dataframe `debar_clean` and print the shape and # of unique employer names

In [64]:
## Part A

# Add "is_dup" column
mult_debar_wide.loc[:, "is_dup"] = mult_debar_wide["start_date_viol1"] == mult_debar_wide["start_date_viol2"]

# Add "violnum" column
debar["violnum"] = debar.groupby(["Name"]).cumcount() + 1

## Part B

# Merge mult_debar_wide with debar 
mult_debar_wide_vn = mult_debar_wide.merge(debar, on=["Name", "City, State"], how="left")

# Filter mult_debar_wide for is_dup == True and violnum == viol1
filtered_employers_df = mult_debar_wide_vn[(mult_debar_wide_vn["is_dup"] == True)]

# Extract the unique employer names
unique_employer_names = filtered_employers_df["Name"].unique()

# Filter debar for employers in unique_employer_names & "viol_num" == 1
debar_clean_pt1 = debar[(debar["Name"].isin(unique_employer_names)) & (debar["violnum"] == 1)]
debar_clean_pt2 = debar[~debar["Name"].isin(unique_employer_names)]

# concatenate the two dfs
debar_clean = pd.concat([debar_clean_pt1, debar_clean_pt2], axis=0)

# Drop the "is_repeated" column from debar_clean
debar_clean = debar_clean.drop(columns=["is_repeated"])
debar_clean = debar_clean.drop_duplicates()
debar_clean.shape





(101, 7)

# 2. Merging and regex (17 points total)



## 2.1 Load data on job postings

The previous dataset contains a small subset of employers who faced temporary bans due to violations of H-2A program regulations

Since most of the bans have expired, we're going to see which of those employers posted new H-2A jobs in the first quarter of 2021 

Loading the `jobs_clean.csv` data stored in `pset4_inputdata`

In [67]:
jobs = pd.read_csv("jobs.csv")
jobs


,CASE_NUMBER,CASE_STATUS,RECEIVED_DATE,DECISION_DATE,TYPE_OF_EMPLOYER_APPLICATION,H2A_LABOR_CONTRACTOR,NATURE_OF_TEMPORARY_NEED,EMERGENCY_FILING,EMPLOYER_NAME,TRADE_NAME_DBA,...,ADDENDUM_B_HOUSING_ATTACHED,TOTAL_HOUSING_RECORDS,MEALS_PROVIDED,MEALS_CHARGED,MEAL_REIMBURSEMENT_MINIMUM,MEAL_REIMBURSEMENT_MAXIMUM,PHONE_TO_APPLY,EMAIL_TO_APPLY,WEBSITE_TO_APPLY,TOTAL_ADDENDUM_A_RECORDS
0,H-300-20199-721302,Determination Issued - Withdrawn,2020-07-17 14:50:40.840,2020-10-01 00:00:00.000,Individual Employer,N,Seasonal,Y,"Fazio Farms Operating Company, LLC",NaN,...,N,1,Y,12.68,12.68,55.0,13607017661,faziofarms@gmail.com,NaN,0
1,H-300-20231-773906,Determination Issued - Certification,2020-08-20 10:38:15.620,2020-10-01 00:00:00.000,Association - Agent,N,Seasonal,N,Charlie Sunderland,Panter & Sunderland Nursery,...,N,1,N,NaN,12.68,55.0,19318083783,NaN,https://www.jobs4tn.gov/vosnet/Default.aspx,0
2,H-300-20231-774123,Determination Issued - Certification,2020-08-24 15:33:14.340,2020-10-01 00:00:00.000,Individual Employer,N,Seasonal,N,Michael Rudebusch,NaN,...,N,1,N,NaN,12.68,55.0,19369333827,fayethlynpitre@rocketmail.com,NaN,0
3,H-300-20231-774151,Determination Issued - Certification,2020-08-21 12:08:09.760,2020-10-01 00:00:00.000,Individual Employer,N,Seasonal,N,Lodahl Farms,NaN,...,Y,2,N,NaN,12.68,55.0,14069637560,lodahl_kelsey@yahoo.com,NaN,0
4,H-300-20231-774508,Determination Issued - Certification,2020-08-20 10:17:34.530,2020-10-01 00:00:00.000,Individual Employer,Y,Seasonal,N,"Dunson Harvesting, Inc.","Dunson Harvesting, Inc.",...,Y,8,N,NaN,12.68,55.0,18632939888,NaN,www.employflorida.com,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2715,H-300-20351-963307,Determination Issued - Certification,2020-12-17 10:34:09.530,2020-12-31 00:00:00.000,Individual Employer,N,Seasonal,N,James L Schneller,NaN,...,N,1,N,NaN,12.68,55.0,12709914373,jims4463@aol.com,NaN,0
2716,H-300-20351-963399,Determination Issued - Certification,2020-12-17 14:37:57.920,2020-12-31 00:00:00.000,Individual Employer,N,Seasonal,N,Stephen G Myers,NaN,...,N,1,N,NaN,12.68,55.0,12708780028,vjpm76@gmail.com,https://kentucky.gov/employment/Pages/default....,1
2717,H-300-20351-964097,Determination Issued - Certification,2020-12-22 12:21:32.370,2020-12-31 00:00:00.000,Individual Employer,N,Seasonal,N,Andy Povey,Andy Povey Farms,...,Y,2,Y,12.68,12.68,55.0,NaN,H-2AJobs@snakeriverfarmers.org,https://idahoworks.gov/ada/r/job_seeker,0
2718,H-300-20351-965435,Determination Issued - Certification,2020-12-22 12:18:43.280,2020-12-31 00:00:00.000,Individual Employer,N,Seasonal,N,"Silver Creek Seed, LLC",NaN,...,Y,2,Y,12.68,12.68,55.0,NaN,H-2AJobs@snakeriverfarmers.org,https://idahoworks.gov/ada/r/job_seeker,0


##  2.2 Try inner join on employer name  (2 points)

- Use the `EMPLOYER_NAME` field of the `jobs` dataset
- Use the `Name` field of the `debar_clean` dataset 

A. Use pd.merge with an inner join on those fields to see whether there are any exact matches. 

B. If there are exact matches, print the row(s) with exact matches



In [78]:
## Part A:
matches = debar_clean.merge(jobs, how = "inner", left_on = "Name", right_on = "EMPLOYER_NAME")

## Part B:
print(matches)

             Name       City, State                         Violation  \
0  RAFAEL BARAJAS  Sebring, Florida  Non-payment of certification fee   

  Duration Start date   End date  violnum    name_clean_x  name_clean_2_x  \
0   1 year  9/23/2016  9/22/2017        1  RAFAEL BARAJAS  RAFAEL BARAJAS   

          CASE_NUMBER  ... MEALS_PROVIDED MEALS_CHARGED  \
0  H-300-20287-876656  ...              Y         12.68   

  MEAL_REIMBURSEMENT_MINIMUM MEAL_REIMBURSEMENT_MAXIMUM PHONE_TO_APPLY  \
0                      12.68                       55.0    18632732686   

  EMAIL_TO_APPLY               WEBSITE_TO_APPLY TOTAL_ADDENDUM_A_RECORDS  \
0            NaN  https://seasonaljobs.dol.gov/                        7   

     name_clean_y  name_clean_2_y  
0  RAFAEL BARAJAS  RAFAEL BARAJAS  

[1 rows x 149 columns]


## 2.3 Targeted regex (10 points total)

You want to see if you can increase the exact match rate with some basic cleaning of each 
of the employer name fields in each dataset 

### 2.3.1 Converting to upper (2 points)

A. Convert the `EMPLOYER_NAME` and `Name` fields to uppercase using list comprehension rather than df.varname.str.upper() (it's fine to do a separate list comprehension line for each of the two columns)

B. Print a random sample of 15 values of each result

C. Assign the full vector of uppercase names back to the original data, writing over the original `EMPLOYER_NAME` and `Name` columns 


In [69]:
## Part A: 
jobs["EMPLOYER_NAME"] = [name.upper() for name in jobs["EMPLOYER_NAME"]]
debar_clean["Name"] = [name.upper() for name in debar_clean["Name"]]


In [70]:
## Part B
print(random.sample(jobs["EMPLOYER_NAME"].tolist(), 15))
print(random.sample(debar_clean["Name"].tolist(), 15))

['RANCHO NUEVO HARVESTING, INC.', 'CA SKILES FARMS, INC.', "OLSON'S GREENHOUSE GARDENS, INC.  ", 'J LINK AQUAFARMS LLC', 'COOPER PLANTING COMPANY PTR', 'BALTAZAR GARCIA HARVESTING & GROVE CARE, INC.', 'DURST FARMS INC.', "LATHAM'S NURSERY, INC.", "PLANT'S & TREES OF TEXAS", 'HENDERSON FARMS', 'CHRIS MOUNIER FARMS', 'BEEF & BACON DRIVE INC. ', 'CARONA FARMS LLC', 'JAMES P. HUNDLEY', 'SAMUEL ONGSTAD']
['TURNER FARMS', 'AB RANCH', 'CISCO PRODUCE INC.', 'TRUST NURSERY', 'YOLANDA CHAVEZ FARMING', 'LEONARD SMITH FARMS', 'ROBERT D. TOWLES', 'K W ENTERPRISES', 'SHARON MATHIS', 'EVERGREEN PRODUCE', 'JOHN & NETA LEOPKY FARMS', 'CIRA CORTEZ LOPEZ', 'E.V. RANCH LLP', 'COUNTY FAIR FARM (COMPANY) AND ANDREW WILLIAMSON (INDIVIDUAL)*', 'ALTENDORF TRANSPORT INC.']


In [57]:
## Part C

# Already done in Part A

#jobs["EMPLOYER_NAME"] = [name.upper() for name in jobs["EMPLOYER_NAME"]]
#debar_clean["Name"] = [name.upper() for name in debar_clean["Name"]]




### 2.3.2 Cleaning up punctuation (4 points)

You notice that INC, CO, and LLC are sometimes followed by a period (.) but sometimes not

A. For each dataset, write a regex pattern using `re.sub` to remove the . but only if it's preceded by INC, LLC, or CO 

Make sure LLC, INC, CO remain part of the string but just without the dot

B. Test the pattern on the positive and negative example we provide below and print the result. See the Github issue for examples of what to return


**Hint**: https://stackoverflow.com/questions/7191209/python-re-sub-replace-with-matched-content



In [73]:
pos_example_1 = "CISCO PRODUCE INC."
pos_example_2 = "AVOYELLES HONEY CO., LLC"
neg_example = "E.V. RANCH LLP"

In [74]:

## Part A 
regex = r"(\b(?:INC|LLC|CO))\."

## Part B
result = re.sub(regex, r'\1', pos_example_1)
result

result_1 = re.sub(regex, r'\1', pos_example_2)
result_1

result_2 = re.sub(regex, r'\1', neg_example)
result_2


'CISCO PRODUCE INC'

'AVOYELLES HONEY CO, LLC'

'E.V. RANCH LLP'

### 2.3.3 (4 points)

Use that pattern in conjunction with `re.sub` and list comprehension to clean the employer name columns in each dataset. Save the new columns as `name_clean` in each. Then, use row subsetting to (1) subset to rows that changed names and (2) for:

- `debar_clean` print the `Name` and `name_clean` columns
- `jobs` print the `EMPLOYER_NAME` and `name_clean` columns

Make sure to use the uppercase versions of the variables


In [75]:
def clean_employer_name(name):
    return re.sub(regex, r'\1', name)

# Clean employer name columns and save as "name_clean"
debar_clean["name_clean"] = [clean_employer_name(name) for name in debar_clean["Name"]]
jobs["name_clean"] = [clean_employer_name(name) for name in jobs["EMPLOYER_NAME"]]

# Subsetting 

# (1) Subset to rows that changed names
debar_changed_names = debar_clean[debar_clean["Name"] != debar_clean["name_clean"]]
jobs_changed_names = jobs[jobs["EMPLOYER_NAME"] != jobs["name_clean"]]

# (2) print columns
print(debar_clean[["Name", "name_clean"]])
print(jobs[["EMPLOYER_NAME", "name_clean"]])


                                                  Name  \
6                              ANNABELLA LAND & CATTLE   
7                                  AUTUMN HILL ORCHARD   
8                CADDO CREEK RANCH, DBA PARADISE RANCH   
11                               LOEWEN HARVESTING LLC   
12                         ROLLO FARM LABOR CONTRACTOR   
..                                                 ...   
107                                       WALKER PLACE   
108  COUNTY FAIR FARM (COMPANY) AND ANDREW WILLIAMS...   
110                                      JESUS LEDESMA   
112        B & R HARVESTING AND PAUL CRUZ (INDIVIDUAL)   
113                                        DELIA ROJAS   

                                            name_clean  
6                              ANNABELLA LAND & CATTLE  
7                                  AUTUMN HILL ORCHARD  
8                CADDO CREEK RANCH, DBA PARADISE RANCH  
11                               LOEWEN HARVESTING LLC  
12                

In [76]:
debar_clean.head()
jobs.head()

,Name,"City, State",Violation,Duration,Start date,End date,violnum,name_clean
6,ANNABELLA LAND & CATTLE,"Annabella, UT",Non Payment,1 year,5/9/2014,5/9/2015,1,ANNABELLA LAND & CATTLE
7,AUTUMN HILL ORCHARD,"Groton, MA",Failure to respond to audit (no response),2 years,7/6/2014,7/5/2016,1,AUTUMN HILL ORCHARD
8,"CADDO CREEK RANCH, DBA PARADISE RANCH","Caddo, TX",Failure to respond to audit (partial response),2 years,7/20/2014,7/19/2016,1,"CADDO CREEK RANCH, DBA PARADISE RANCH"
11,LOEWEN HARVESTING LLC,"Brownfield, TX",Failure to respond to audit (partial response),1 year,8/20/2014,8/19/2015,1,LOEWEN HARVESTING LLC
12,ROLLO FARM LABOR CONTRACTOR,"Miami, FL",Failure to respond to audit (no response),2 years,8/23/2014,8/22/2016,1,ROLLO FARM LABOR CONTRACTOR


,CASE_NUMBER,CASE_STATUS,RECEIVED_DATE,DECISION_DATE,TYPE_OF_EMPLOYER_APPLICATION,H2A_LABOR_CONTRACTOR,NATURE_OF_TEMPORARY_NEED,EMERGENCY_FILING,EMPLOYER_NAME,TRADE_NAME_DBA,...,TOTAL_HOUSING_RECORDS,MEALS_PROVIDED,MEALS_CHARGED,MEAL_REIMBURSEMENT_MINIMUM,MEAL_REIMBURSEMENT_MAXIMUM,PHONE_TO_APPLY,EMAIL_TO_APPLY,WEBSITE_TO_APPLY,TOTAL_ADDENDUM_A_RECORDS,name_clean
0,H-300-20199-721302,Determination Issued - Withdrawn,2020-07-17 14:50:40.840,2020-10-01 00:00:00.000,Individual Employer,N,Seasonal,Y,"FAZIO FARMS OPERATING COMPANY, LLC",NaN,...,1,Y,12.68,12.68,55.0,13607017661,faziofarms@gmail.com,NaN,0,"FAZIO FARMS OPERATING COMPANY, LLC"
1,H-300-20231-773906,Determination Issued - Certification,2020-08-20 10:38:15.620,2020-10-01 00:00:00.000,Association - Agent,N,Seasonal,N,CHARLIE SUNDERLAND,Panter & Sunderland Nursery,...,1,N,NaN,12.68,55.0,19318083783,NaN,https://www.jobs4tn.gov/vosnet/Default.aspx,0,CHARLIE SUNDERLAND
2,H-300-20231-774123,Determination Issued - Certification,2020-08-24 15:33:14.340,2020-10-01 00:00:00.000,Individual Employer,N,Seasonal,N,MICHAEL RUDEBUSCH,NaN,...,1,N,NaN,12.68,55.0,19369333827,fayethlynpitre@rocketmail.com,NaN,0,MICHAEL RUDEBUSCH
3,H-300-20231-774151,Determination Issued - Certification,2020-08-21 12:08:09.760,2020-10-01 00:00:00.000,Individual Employer,N,Seasonal,N,LODAHL FARMS,NaN,...,2,N,NaN,12.68,55.0,14069637560,lodahl_kelsey@yahoo.com,NaN,0,LODAHL FARMS
4,H-300-20231-774508,Determination Issued - Certification,2020-08-20 10:17:34.530,2020-10-01 00:00:00.000,Individual Employer,Y,Seasonal,N,"DUNSON HARVESTING, INC.","Dunson Harvesting, Inc.",...,8,N,NaN,12.68,55.0,18632939888,NaN,www.employflorida.com,4,"DUNSON HARVESTING, INC"


## 2.4 More joins and more cleaning (5 points)

A. Conduct another inner join between `jobs` and `debar_clean` now using the `name_clean` column; print the result. Did the cleaning result in any more employers matched between the two datasets?

B. Create a new column in `debar_clean` called `name_clean_2` that uses regex to take the following name in that dataset:

- `SLASH E.V. RANCH LLP` in the `debar_clean` dataset

And cleans it up so that it matches with this employer in `jobs`

- `SLASH EV RANCH` in the `jobs` dataset

Eg a pattern to remove the dots in the EV and the space+LLP-- you can apply the pattern to all employer names in debar_clean (so don't need to worry about only applying it to that one employer)


C. Conduct a left join using `name_clean_2` as the join column where the left hand dataframe is `jobs`; right hand dataframe is `debar_clean`, store the result as a dataframe, and print the rows where the merge indicator indicates the row was found in both dataframe

**Note**: this manual cleaning process is inefficient and helps motivate why talked about fuzzy matching. Fuzzy matching could recognize that Slash EV ranch is a highly similar string to slash ev ranch llp and match them without us needing to use regex to make the strings identical.

In [77]:
## Part A:

jobs.merge(debar_clean, on = "name_clean", how = "inner")

## Part B:

# Define the regex to remove dots and " LLP"
regex_2 = r'\.|\sLLP'

# Function to clean each name
def clean_name(name):
    return re.sub(regex_2, '', name)

# Create "name_clean_2" in debar_clean
debar_clean["name_clean_2"] = debar_clean["Name"].apply(clean_name)

# Create "name_clean_2" in jobs
jobs["name_clean_2"] = jobs["EMPLOYER_NAME"].apply(clean_name)

## Part C:

# merge jobs and debar_clean on "name_clean_2"
jobs_debar_clean_nc2 = jobs.merge(debar_clean, how = "left", on = "name_clean_2", indicator = True)

# print rows found in both dataframes
both_found_rows = jobs_debar_clean_nc2[jobs_debar_clean_nc2["_merge"] == "both"]
print(both_found_rows)




,CASE_NUMBER,CASE_STATUS,RECEIVED_DATE,DECISION_DATE,TYPE_OF_EMPLOYER_APPLICATION,H2A_LABOR_CONTRACTOR,NATURE_OF_TEMPORARY_NEED,EMERGENCY_FILING,EMPLOYER_NAME,TRADE_NAME_DBA,...,WEBSITE_TO_APPLY,TOTAL_ADDENDUM_A_RECORDS,name_clean,Name,"City, State",Violation,Duration,Start date,End date,violnum
0,H-300-20287-876656,Determination Issued - Certification,2020-10-20 09:20:32.010,2020-11-09 00:00:00.000,Individual Employer,Y,Seasonal,Y,RAFAEL BARAJAS,NaN,...,https://seasonaljobs.dol.gov/,7,RAFAEL BARAJAS,RAFAEL BARAJAS,"Sebring, Florida",Non-payment of certification fee,1 year,9/23/2016,9/22/2017,1


             CASE_NUMBER                           CASE_STATUS  \
791   H-300-20287-876656  Determination Issued - Certification   
1115  H-300-20306-894148  Determination Issued - Certification   

                RECEIVED_DATE            DECISION_DATE  \
791   2020-10-20 09:20:32.010  2020-11-09 00:00:00.000   
1115  2020-11-02 18:11:29.140  2020-11-24 00:00:00.000   

     TYPE_OF_EMPLOYER_APPLICATION H2A_LABOR_CONTRACTOR  \
791           Individual Employer                    Y   
1115          Individual Employer                    N   

     NATURE_OF_TEMPORARY_NEED EMERGENCY_FILING   EMPLOYER_NAME TRADE_NAME_DBA  \
791                  Seasonal                Y  RAFAEL BARAJAS            NaN   
1115                 Seasonal                N  SLASH EV RANCH            NaN   

      ...    name_clean_2                  Name       City, State  \
791   ...  RAFAEL BARAJAS        RAFAEL BARAJAS  Sebring, Florida   
1115  ...  SLASH EV RANCH  SLASH E.V. RANCH LLP         Rifle, CO   


# 3. Optional extra credit 1: regex to separate companies from individuals (1 point)

You notice some employers in `debar_clean` have both the name of the company and the name of individual, e.g.:
    
COUNTY FAIR FARM (COMPANY) AND ANDREW WILLIAMSON (INDIVIDUAL)*

Use the uppercase/cleaned `name_clean` in `debar_clean`

A. Write a regex pattern that does the following:
    - Captures the pattern that occurs before COMPANY if (COMPANY) is in string; so in example above, extracts COUNTY FAIR FARM 
    - Captures the pattern that occurs before INDIVIDUAL if (INDIVIDUAL) is also in string -- so in above, extracts ANDREW WILLIAMSON (so omit the "and")
    
B. Test the pattern on `pos_example` and `neg_example`-- make sure former returns a list (if using find.all) or match object (if using re.search) with the company name and individual name separated out; make sure latter returns empty
    
**Hints and resources**: for step A, you can either use re.search, re.match, or re.findall; don't worry about matching B&R Harvesting and Paul Cruz (Individual)

- Same regex resources as above
    

In [91]:
pos_example = "COUNTY FAIR FARM (COMPANY) AND ANDREW WILLIAMSON (INDIVIDUAL)*"
neg_example = "CISCO PRODUCE INC"

# Regex to capture the patterns
company_pattern = re.compile(r'(.+?)\s*\(COMPANY\)')
individual_pattern = re.compile(r'\(COMPANY\)\s+AND\s+(.*?)\s+\(INDIVIDUAL\)\*')

# Positive
re.findall(company_pattern, pos_example)
re.findall(individual_pattern, pos_example)

# Negative 
re.findall(company_pattern, neg_example)
re.findall(individual_pattern, neg_example)


['COUNTY FAIR FARM']

['ANDREW WILLIAMSON']

[]

[]

C. Iterate over the `name_clean` column in debar and use regex to create two new columns in `debar_clean`:
   - `co_name`: A column for company (full `name_clean` string if no match; pattern before COMPANY if one extracted)
   - `ind_name`: A column for individual (full `name_clean` string if no match; pattern before INDIVIDUAL if one extracted)
 


In [93]:
# Regex to capture the patterns
company_pattern = re.compile(r'(.+?)\s*\(COMPANY\)')
individual_pattern = re.compile(r'\(COMPANY\)\s+AND\s+(.*?)\s+\(INDIVIDUAL\)\*')

# Create empty lists
co_names = []
ind_names = []

# Iterate over the "name_clean_2" column in "debar_clean"
for name_clean_2 in debar_clean["name_clean_2"]:

    # Search for the company and individual patterns
    company_match = company_pattern.search(name_clean_2)
    individual_match = individual_pattern.search(name_clean_2)

    # Extract company and individual names, or use the original name if no match
    company_name = company_match.group(1) if company_match else name_clean_2
    individual_name = individual_match.group(1) if individual_match else name_clean_2

    # Append the extracted names to the respective lists
    co_names.append(company_name)
    ind_names.append(individual_name)

# Add the lists of company and individual names as new columns to 'debar_clean'
debar_clean["co_name"] = co_names
debar_clean["ind_name"] = ind_names

debar_clean[["co_name", "ind_name"]].tail()

,co_name,ind_name
107,WALKER PLACE,WALKER PLACE
108,COUNTY FAIR FARM,ANDREW WILLIAMSON
110,JESUS LEDESMA,JESUS LEDESMA
112,B & R HARVESTING AND PAUL CRUZ (INDIVIDUAL),B & R HARVESTING AND PAUL CRUZ (INDIVIDUAL)
113,DELIA ROJAS,DELIA ROJAS


   
D. Print three columns for the rows in `debar_clean` containing the negative example and positive example described above (county fair farm and cisco produce):

- `name_clean`
- `co_name`
- `ind_name`
- `Violation`

**Note**: as shown in the outcome there may be duplicates of the same company reflecting different violations

In [94]:
pos_example = "COUNTY FAIR FARM (COMPANY) AND ANDREW WILLIAMSON (INDIVIDUAL)*"
neg_example = "CISCO PRODUCE INC"

# Filter for the positive and negative examples
selected_rows = debar_clean[
    (debar_clean["name_clean_2"] == pos_example) | (debar_clean["name_clean_2"] == neg_example)
]

# Select the four columns
selected_columns = selected_rows[["name_clean_2", "co_name", "ind_name", "Violation"]]

selected_columns

,name_clean_2,co_name,ind_name,Violation
19,CISCO PRODUCE INC,CISCO PRODUCE INC,CISCO PRODUCE INC,Failure to respond to audit (no response)
56,CISCO PRODUCE INC,CISCO PRODUCE INC,CISCO PRODUCE INC,Impeding the Audit Process – Non- Response
108,COUNTY FAIR FARM (COMPANY) AND ANDREW WILLIAMS...,COUNTY FAIR FARM,ANDREW WILLIAMSON,WHD Debarment


# 4. Optional extra credit 2 (up to 3 points)

- For 1 point extra credit, create a visualization with 1+ of the existing fields in either the raw `jobs` or `debar` data. We'll be showing cool visualizations in class so use your imagination! Options could include visualizing between-state or over-time variation

- For 3 points extra credit instead, geocode the employer addresses in `jobs` and plot the addresses of jobs as points overlaid on top of a map of Georgia 
    - **Note**: this extra credit involves Googling since we have not yet covered spatial data. 
        - For discussion of how to geocode addresses -> lat/long, see: https://www.natasshaselvaraj.com/a-step-by-step-guide-on-geocoding-in-python/ 
        - For discussion of plotting lat/long dots against a map, see this discussion of geopandas: https://towardsdatascience.com/plotting-maps-with-geopandas-428c97295a73
    - Relevant columns include `EMPLOYER_ADDRESS_1` 
    - The geocoding might have a long runtime so feel free to implement it in a separate .py script that you submit alongside your notebook and to just read in the geocoded data

In [125]:
# Convert the 'date_strings' column to date objects

#date_format = '%m/%d/%y'

#debar['Start date'] = pd.to_datetime(debar["Start date"], format=date_format).dt.date

p = ggplot(debar, aes(x="City, State")) + geom_bar() + theme(axis_text_x=element_text(angle=90, hjust=1))

p.save("4EC.png")

/Users/janehuang/anaconda3/lib/python3.11/site-packages/plotnine/ggplot.py:587: PlotnineWarning: Saving 6.4 x 4.8 in image.
/Users/janehuang/anaconda3/lib/python3.11/site-packages/plotnine/ggplot.py:588: PlotnineWarning: Filename: 4EC.png


In [103]:
debar_clean

,Name,"City, State",Violation,Duration,Start date,End date,violnum,name_clean,name_clean_2,co_name,ind_name
6,ANNABELLA LAND & CATTLE,"Annabella, UT",Non Payment,1 year,5/9/2014,5/9/2015,1,ANNABELLA LAND & CATTLE,ANNABELLA LAND & CATTLE,ANNABELLA LAND & CATTLE,ANNABELLA LAND & CATTLE
7,AUTUMN HILL ORCHARD,"Groton, MA",Failure to respond to audit (no response),2 years,7/6/2014,7/5/2016,1,AUTUMN HILL ORCHARD,AUTUMN HILL ORCHARD,AUTUMN HILL ORCHARD,AUTUMN HILL ORCHARD
8,"CADDO CREEK RANCH, DBA PARADISE RANCH","Caddo, TX",Failure to respond to audit (partial response),2 years,7/20/2014,7/19/2016,1,"CADDO CREEK RANCH, DBA PARADISE RANCH","CADDO CREEK RANCH, DBA PARADISE RANCH","CADDO CREEK RANCH, DBA PARADISE RANCH","CADDO CREEK RANCH, DBA PARADISE RANCH"
11,LOEWEN HARVESTING LLC,"Brownfield, TX",Failure to respond to audit (partial response),1 year,8/20/2014,8/19/2015,1,LOEWEN HARVESTING LLC,LOEWEN HARVESTING LLC,LOEWEN HARVESTING LLC,LOEWEN HARVESTING LLC
12,ROLLO FARM LABOR CONTRACTOR,"Miami, FL",Failure to respond to audit (no response),2 years,8/23/2014,8/22/2016,1,ROLLO FARM LABOR CONTRACTOR,ROLLO FARM LABOR CONTRACTOR,ROLLO FARM LABOR CONTRACTOR,ROLLO FARM LABOR CONTRACTOR
...,...,...,...,...,...,...,...,...,...,...,...
107,WALKER PLACE,"Danville, IL",Failure to comply with the employer's obligati...,2 months,11/19/2019,1/26/2020,1,WALKER PLACE,WALKER PLACE,WALKER PLACE,WALKER PLACE
108,COUNTY FAIR FARM (COMPANY) AND ANDREW WILLIAMS...,"Jefferson, ME",WHD Debarment,3 years,3/8/2017,3/8/2020,1,COUNTY FAIR FARM (COMPANY) AND ANDREW WILLIAMS...,COUNTY FAIR FARM (COMPANY) AND ANDREW WILLIAMS...,COUNTY FAIR FARM,ANDREW WILLIAMSON
110,JESUS LEDESMA,"Mulberry, FL",Failure to Respond to Audit Request,2 years,2/8/18,2/8/20,1,JESUS LEDESMA,JESUS LEDESMA,JESUS LEDESMA,JESUS LEDESMA
112,B & R HARVESTING AND PAUL CRUZ (INDIVIDUAL),"Collins, GA",WHD Debarment,3 years,4/9/17,4/9/20,1,B & R HARVESTING AND PAUL CRUZ (INDIVIDUAL),B & R HARVESTING AND PAUL CRUZ (INDIVIDUAL),B & R HARVESTING AND PAUL CRUZ (INDIVIDUAL),B & R HARVESTING AND PAUL CRUZ (INDIVIDUAL)
